<a href="https://colab.research.google.com/github/Hamza-benAmmar/PPP_Essai/blob/main/Emotion_Classification_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer
import nltk
import tensorflow as tf
nltk.download('stopwords')
nltk.download('punkt')
!pip install keras-tuner


# Data Classifier

In [ ]:
data=pd.read_csv('/content/concatenated_data.csv')

In [ ]:
data=data.loc[:,['sentiment','content']]
copy=data.copy()
data['sentiment'].value_counts()

In [ ]:
data.isnull().sum()


In [ ]:
data.duplicated().sum()


In [ ]:
index = data[data.duplicated() == True].index
data.drop(index, axis = 0, inplace = True)
data.reset_index(inplace=True, drop = True)

In [ ]:
data.duplicated().sum()


In [ ]:
data[data['content'].duplicated() == True]


In [ ]:
index = data[data['content'].duplicated() == True].index
data.drop(index, axis = 0, inplace = True)
data.reset_index(inplace=True, drop = True)

#  Data Cleaning and Preprocessing


In [ ]:
stop_words=stopwords.words("english")
from nltk.stem import WordNetLemmatizer

def lemmatization(text):
    lemmatizer= WordNetLemmatizer()

    text = text.split()

    text=[lemmatizer.lemmatize(y) for y in text]
    
    return " " .join(text)

def remove_stop_words(text):

    Text=[i for i in str(text).split() if i not in stop_words]
    return " ".join(Text)

def Removing_numbers(text):
    text=''.join([i for i in text if not i.isdigit()])
    return text

def lower_case(text):
    
    text = text.split()

    text=[y.lower() for y in text]
    
    return " " .join(text)

def Removing_punctuations(text):
    ## Remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)
    text = text.replace('؛',"", )
    
    ## remove extra whitespace
    text = re.sub('\s+', ' ', text)
    text =  " ".join(text.split())
    return text.strip()

def Removing_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_small_sentences(df):
    for i in range(len(df)):
        if len(df.text.iloc[i].split()) < 3:
            df.text.iloc[i] = np.nan
            
def normalize_text(data):
    data["content"]=data["content"].apply(lambda text : Removing_urls(text))
    data["content"]=data["content"].apply(lambda text : lower_case(text))
    data["content"]=data["content"].apply(lambda text : remove_stop_words(text))
    data["content"]=data["content"].apply(lambda text : Removing_numbers(text))
    data["content"]=data["content"].apply(lambda text : Removing_punctuations(text)) 
    data["content"]=data["content"].apply(lambda text : lemmatization(text))
    return data



In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
data=normalize_text(data)

In [ ]:
data.tail()

In [ ]:
data['sentiment'].value_counts()


# LABEL ENCODING

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['sentiment_label']=le.fit_transform(data['sentiment'])
data['sentiment_label'].value_counts()


In [ ]:
max_length = 0
for sentence in data['content']:
    length = len(sentence.split())
    if length > max_length:
        max_length = length
max_length

# Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X=data['content']
y=data['sentiment_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y,random_state=42)

# Feauture Engineering

In [ ]:
from keras.preprocessing.text import one_hot, Tokenizer
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train)
X_train = word_tokenizer.texts_to_sequences(X_train)
X_test = word_tokenizer.texts_to_sequences(X_test) 


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 35
X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
X_train.shape

In [ ]:
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length


In [ ]:
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/glove.twitter.27B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    try:
      vector_dimensions = asarray(records[1:], dtype='float32')
      embeddings_dictionary[word] = vector_dimensions
    except ValueError:
      pass

glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    try:
      if embedding_vector is not None:
          embedding_matrix[index] = embedding_vector
    except ValueError:
      print(embedding_vector,word,index)
      pass


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense,LSTM,Lambda
from keras.utils import to_categorical
from keras.layers import Bidirectional, Dropout
from keras import regularizers

y_one_hot = to_categorical(y_train, num_classes=6)
y_one_hot



In [ ]:
y_one_hot

# Model Building Phase



In [ ]:
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Embedding(vocab_length, 100, input_length=X_train.shape[1], weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(256, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2, return_sequences=True)))
model.add(Bidirectional(LSTM(128, dropout=0.2,recurrent_dropout=0.2)))
model.add(Dense(6, activation='softmax'))
                                                                                         
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.005), metrics=['accuracy'])
model.summary()

In [ ]:

history = model.fit(X_train,
                    y_one_hot,
                    validation_split=0.2,
                    verbose=1,
                    batch_size=256,
                    epochs=6
                   )

In [ ]:
import numpy as np
y_train = np.array(y_train)

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
 
# Plot training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'], loc='upper right')
plt.show()



# Prediction


In [ ]:
from sklearn.metrics import accuracy_score

y_pred=model.predict(X_test)

y_pred

In [ ]:
y_pred = [list(i).index(max(i)) for i in y_pred]


In [ ]:
type(y_pred)

In [ ]:
y_list=list(y_test)

In [ ]:
print(accuracy_score(y_test, y_pred))

# Classification Metrics


In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, output_dict=True)
print(classification_report(y_test, y_pred))

In [ ]:



# Generate classification report

# Extract precision, recall, and F1-score from the report
classes = list(report.keys())[:-3]
custom_labels = ['Anger', 'Fear', 'Joy', 'Love', 'Sadness', 'Surprise']
precision = [report[class_name]['precision'] for class_name in classes]
recall = [report[class_name]['recall'] for class_name in classes]
f1_score = [report[class_name]['f1-score'] for class_name in classes]

# Plotting the results
x = np.arange(len(classes))
width = 0.3

fig, ax = plt.subplots()
rects1 = ax.bar(x - width, precision, width, label='Precision')
rects2 = ax.bar(x, recall, width, label='Recall')
rects3 = ax.bar(x + width, f1_score, width, label='F1-Score')

ax.set_ylabel('Score')
ax.set_title('LSTM : Classification Metrics')
ax.set_xticks(x)
ax.set_xticklabels(custom_labels)
ax.legend()

plt.show()


# TEST

In [ ]:
def normalized_sentence(sentence):
    sentence= lower_case(sentence)
    sentence= remove_stop_words(sentence)
    sentence= Removing_numbers(sentence)
    sentence= Removing_punctuations(sentence)
    sentence= Removing_urls(sentence)
    sentence= lemmatization(sentence)
    return sentence

In [ ]:
sentences = [
            "i'm really scared",
            "oh congratulations !",
            "I can't do it, I'm not ready to lose anything, just leave me alone",
            "i am really sad"
            ]
for sentence in sentences:
    print(sentence)
    sentence = normalized_sentence(sentence)
    sentence = word_tokenizer.texts_to_sequences([sentence])
    sentence = pad_sequences(sentence, maxlen=35, truncating='pre')
    result = le.inverse_transform(np.argmax(model.predict(sentence), axis=-1))[0]
    proba =  np.max(model.predict(sentence))
    print(f"{result} : {proba}\n\n")

In [ ]:
#i guess it is stupid Hahahah